In [5]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [6]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [7]:
from helpers_queries import queries
assets = None
with engine.begin() as conn:
  assets = run_sql(conn, queries['list_assets_longterm_bars']).mappings().all()
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'bars_w': 201, 'bars_d': 1337},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'bars_w': 127, 'bars_d': 819},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'bars_w': 220, 'bars_d': 1470},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'bars_w': 101, 'bars_d': 636},
 {'symbol': 'CI', 'exchange': 'NYSE', 'bars_w': 132, 'bars_d': 854},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'bars_w': 222, 'bars_d': 1484},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'bars_w': 49, 'bars_d': 273},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'bars_w': 221, 'bars_d': 1477},
 {'symbol': 'DBK', 'exchange': 'XETR', 'bars_w': 122, 'bars_d': 784},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'bars_w': 142, 'bars_d': 924},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'bars_w': 94, 'bars_d': 588},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'bars_w': 123, 'bars_d': 791},
 {'symbol': 'INMD', 'exchange': 'NASDAQ', 'bars_w': 238, 'bars_d': 1596},
 {'symbol': 'JPM', 'exchange': 'NYSE', 'bars_w': 137, 'bars_d': 

In [8]:
from helpers_db import insert_on_conflict_nothing_ticks

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, interval=Interval.in_daily, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=interval,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

# for asset in assets:
symbol    = 'AMZN' # asset['symbol']
exchange  = 'NASDAQ' # asset['exchange']
table     = 'ticks_1w' # 'ticks_1d'
print(f"Processing {exchange}:{symbol}")
try:
  bars      = 171 # 10 # asset['bars_d']
  interval  = Interval.in_4_hour
  df        = fetch_ticks(symbol=symbol, exchange=exchange, bars=bars, interval=interval) # 7*24
  df        = normalize_dataframe(df)
  df.to_sql(table, engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  df.to_dict
  # df.to_sql('ticks_1w', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  print(f"  count: {df.shape[0]}")
except Exception as e:
  print(f"Exception {exchange}:{symbol}")
  print(str(e))


Processing NASDAQ:AMZN
  count: 171
